# Air Quality Prediction for Major Cities of China $-$ Final Report

## Executive Summary

Understanding and predicting air pollution have become increasingly important to people at all levels in China over the past few years. Hourly PM2.5 data of five major Chinese cities in the period of 2010-2015 are acquired, analyzed and modeled in this project, to identify PM2.5 patterns and forecast future air quality: <br>

**PM2.5 Patterns**<br>
* alters siginificantly among cities: air quality rankings from the best to the worse are Guangzhou, Shanghai, Chengdu and Shenyang, Beijing. Unfortunately, PM2.5 levels for all five cities are too high to be considered healthy and safe according to EPA standards.<br>
* varies systematically with hour of the day, month of the year and from year to year: daily and monthly trends are unique for individual cities, but yearly trend from 2010 to 2015 shows a consistent improvement in all cities.<br>
* correlates with meteorological weather data to some extent: both high winds and large precipitations help to improve air quality in all cities; wind direction also affect air quality, but the impact differs by city, as determined by city location and neighbour environment; air quality are positively correlated to temperature and dew point, but negatively correlated to pressure and humidity.<br>

**Modeling PM2.5**<br>
Modeling and predicting air quality (PM2.5) is a complex problem, as there are many underlying factors all playing an role in shaping PM2.5. As revealed, both historical values, datetime and various weather conditions all have impacts on future PM2.5 values. As the winning model suggests, historical value is crucial in setting the baseline of the next step, and various meteorological weather conditions as well as datetime features determines how PM2.5 value in the next step deviates from its previous step. The optimized model not only forecasts into the future, but also extends to predict an unseen city.<br>

## 1. Introduction

### 1.1 The Problem

<p>In recent years, air pollution has become an increasingly severe problem in China due to rapid industrialization and high energy consumption. The thick haze and smog, characteristic of unhealthy air condition, raise intense public concerns and media attentions, as it has been linked to the increased occurrence of various respiratory diseases and resident mortality rates. According to reports, PM2.5 (ultrafine particles less than 2.5 microns in diameter) is the main pollutant accountable for at least two thirds of the severely polluted days in Chinese major cities, posting a significant threat to human health.</p>
<p>China has started the PM2.5 monitoring and reporting program in major cities since early 2013, and has extended to 388 cities by 2015. In addition, PM2.5 data prior to 2013 are available from US embassy in major Chinese cities. However, besides live monitoring and reporting, understanding PM2.5 trends and building a PM2.5 prediction system are strongly desired. With predictions, government and citizens can plan ahead and take actions accordingly.</p>

### 1.2 The Approach

<p>In this project, we study the PM2.5 trends and its correlation to a variety of weather conditions, using the public datasets by UCI Repository containing hourly PM2.5 measurements for five major Chinese cities along with many other meteorological readings collected during the period of 2010-2015. Graphical and statistifcal EDA are performed to understand the time trends of PM2.5 variation, cross-city comparison of PM2.5 diversity and its correlation to a variety of weather conditions, such as temperature, humidity, pressure, wind direction and speed, precipitation, etc.  Predictive machine learning models are constructed and optimized for predicting future PM2.5 levels following three different methodologies, with the best-performing models achieving $R^2$ scores of 0.94.</p>

### 1.3 The Client

<p>A PM2.5 predictive model is beneficial to people at all levels in China, ranging from Chinese government, local organizations to individual citizens. Government can use this model to predict air quality, provide outdoor activity guideline for adults and children and issue appropriate warnings in advance. Local organizations can benefit from the model prediction to plan outdoor events, schedule emergency facility shutdowns when necessary. Individual citizens can also consult this predictive model to plan their commute choices, better schedule their daily outdoor activities to take full advantage of the clear sky hours and wear protective mask and clothings. The predictive model with reasonable accuracy will enable people to plan ahead accordingly, reducing the hassles and economical loss due to last minute changes as well as avoiding the exposure to extremely unhealthy air.</p>

### 1.4 The Dataset
<p>PM2.5 data are acquired from UCI Machine Learning Repository at <br> https://archive.ics.uci.edu/ml/datasets/PM2.5+Data+of+Five+Chinese+Cities and <br> https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data. <br> The first dataset contains hourly PM2.5 measurements for five Chinese cities (Beijing included), along with many other meteorological readings collected during the period of 2010-2015. These meteorological readings include dew point, temperature, humidity, pressure, combined wind direction, cumulated wind speed and hourly precipitation etc. The second dataset contains similar measurements for city Beijing alone during the period of 2010-2014. In this project, we focus on the first dataset, but also taking into consideration the ‘cumulative hours of snow’ and ‘cumulative hours of rain’ for city Beijing from the second dataset.</p>

## 2. Data Wrangling

<p>The raw data are downloaded as six separate CSV files, imported and concatenated as a single Pandas Dataframe. The concatenated raw dataset is composed of 306744 rows and 30 columns, containing a combination of time-related records and various measurements taken at multiple stations in each city for a total of five cities in China from year 2010 to 2015. The raw dataframe is divided into three separate subsets for wrangling, EDA and modeling, and three new columns are introduced to the appropriate subsets:</p>
    
<li>**Three separate subsets:**<br>
    **pm_clean:** this is the main dataset containing the measurement data for all five cities, here the average pm2.5 reading for each city is used instead of pm2.5 data from individual stations; this subset containing the station_average pm2.5 reading is used for building predictive models.<br>
    **pm_stations:** this is a supplementary dataset containing station-specific individual PM2.5 readings, serving the purpose of validating the measurement consistency among stations in each city, which is the underlying foundation for taking the average pm2.5 reading across multiple stations.<br>
    **pm_sr:** this is also a side dataset containing information on cumulated hours of snows and rains for city 'beijing'. This dataset simply serves the purpose of comparing the impact of snow versus rain on PM2.5 readings.<br>

<li>**Three added columns:**<br> 
    **'date_time':** time-related information is recorded in separated columns as 'year', 'month', 'day', 'hour' and 'season' in the raw data. For EDA and modeling purposes, a DateTime formatted column is created by parsing the time-related columns and added to the 'pm_clean' and 'pm_sr' subsets.<br>
    **'pm_average':** Hourly PM2.5 readings from multiple stations are recorded for each city in the raw data. The pm2.5 readings are reasonably consistent among stations in the same city and there's no reason to choose the PM2.5 reading from one station over others, therefore it's most representative to use the average PM2.5 readings in EDA and Modeling. A column 'pm_average' representing the average of pm2.5 readings from multiple stations of the same city is computed and used in the cleaned main dataset 'pm_clean' instead of the PM2.5 readings from individual station.<br>
    **'ws':** The 'iws' column records the cumulated wind speed over time, one of the meterological weather parameters. However the numbers in this column contain periodically reset to zero when wind direction alters, causing undesired artifacts. Therefore, it makes better sense to extract the hourly wind speed data ('ws') from cumulated wind speed ('iws') and use 'ws' for analysis and modeling instead of 'iws'.

<li>**Missing data:**<br>
There are a total of 36% rows containing missing data, 93% of which are due to the missing pm2.5 values. Given that pm2.5 (air quality indicator) is the main focus for the EDA and predictive modeling, records without valid pm2.5 values are of little use to this study. These missing data are dropped.
    
<li>**Outliers:**<br>
Unrealistic values (0.025%) are spotted in three columns ('dewp', 'humi' and 'ws'). These outliers are either replaced by nearby records when appropriate or dropped.
    
<p>The cleaned data are stored in three separate dataframes, pm_clean, pm_stations and pm_sr. A short description for each dataframe is shown below: 
<img src="figures/columnnotation.png">

More details and source codes on Data Wrangling are available at: <br>
https://github.com/jingzhaomirror/Springboard-Capstone-Project-1/blob/master/milestone_report.ipynb <br>
https://github.com/jingzhaomirror/Springboard-Capstone-Project-1/blob/master/data_wrangling.ipynb

## 3. EDA

### 3.1 PM2.5 consistency across stations

First things first, the inter-station comparison are performed to verify the air quality measurement (PM2.5) consistency among stations in the same city. This consistency check lays the foundation for utilizing the station-average PM2.5 levels instead of the station-specific PM2.5 readings in all other EDA and modeling. 

Shown below is the inter-station comparison for city Beijing as an example, while comparison results for the other four cities are fairly consistent as well. 

<img src="figures/pm_consistency_across_stations_in_beijing.svg">

### 3.2 Air quality, how bad is it and how does it differ among cities?
Air quality in metropolitan areas of China has raised serious concerns in the past few years. To understand how severe the problem is and how it varies by city, daily and annual PM2.5 statistics over a time span of at least 3 years are plotted alongside EPA standards for all five cities. 

<img src="figures/hourly_pm_diversity_among_cities.svg">

<p>According to the United States Environmental Protection Agency (EPA), EPA has established PM2.5 standards that specifically addressed particulate matter smaller than 2.5 microns (PM2.5) since 1997. According to the latest revisions by EPA, the national 24-hour PM2.5 standard (daily average) is 35 μg/m3 and below (September, 2006) whereas the annual PM2.5 standard (yearly average) is 12μg/m3 and below (December, 2012).</p>

<p>As the above comparison shows, all five cities fail to meet the EPA standards:
    <li>Most of the daily average PM2.5 levels (data points in red) exceeds the EPA recommendation for 24-hour PM2.5 level;
    <li>None of the annual PM2.5 levels meet the EPA guidance for annual PM2.5 level, not even the yearly minimum PM2.5 levels;<br>
        
This reveals the truly problematic air conditions in densely populated cities of China. PM2.5 levels are too high to be considered healthy and safe according to EPA standards. Although actions and regulations have been taken to address the problem, the impacts are still far from satisfactory. In fact, as the data implies, although PM2.5 levels are gradually improving over time in all five cities, the progress can only be considered moderate. Learning from the data and history (lessons from London, Los Angeles, etc.), the problem takes years and efforts from all levels to resolve. 

In addition to the daily and yearly PM2.5 plots, Cumulative Density Function (CDF) and box-and-whisker plot are also created to provide a more direct quantitative comparison of air quality (hourly PM2.5 over the entire time span) among cities. 

<img src="figures/cdf_boxplot_pm_diversity_among_cities.svg">

Combining all three types of plots, the following findings on PM2.5 variation by city can be drawn:
    <li>Air quality ranking from the best to the worst are: Guangzhou, Shanghai, Chengdu/Shenyang, Beijing; 
    <li>Among the five, Guangzhou has the most consistent low PM2.5 levels and the fewest days of high PM2.5 levels, whereas Beijing has the highest average PM2.5 levels and the widest spread of PM2.5 readings;
    <li>Five cities all have distinct PM2.5 spikes, as peak PM2.5 readings can result from many factors, for instance, local temperature, wind, etc. These factors are city specific, therefore PM2.5 spikes are not overlapping; 
    <li>Five cities share similar trends over time: PM2.5 readings are relative higher with larger fluctuations during winter periods and lower with smaller fluctuations in summers;

### 3.3 Air quality trends over time

To understand the hourly, daily, monthly, seasonal and yearly variations of air quality in each city, hourly PM2.5 are plotted vs hour of the day, day of the week, month of the year, season of the year, and as a function of different years. Trends of PM2.5 vs various time frequencies are shown below by city.

<img src="figures/pm_trend_over_time_by_cities.svg">

**The hourly trends follow two distinct patterns corresponding to their unique physical locations** 
* Beijing, Chengdu, Shenyang (inland cities): larger fluctuations during the 24-hour period;
PM2.5 levels rise slightly during morning rush hours, then drop down afterwards; The readings reach the lowest in the afternoon, followed by a bound back during evening hours, and remain high throughout the night;
* Shanghai, Guangzhou (coastal cities): only minor fluctuations during the 24-hour period;
PM2.5 levels remain relatively low throughout the day, with only minor increases during evening hours;

**The monthly trends demonstrate similar patterns:**
* All five cities share somewhat similar variations around the year, with larger fluctuation (quantile ranges) in winter months and smaller fluctuation in summer months. 
* PM2.5 starts to drop during spring months, reaches the lowest during summer months, begins to bound up during fall months and rapidly reaches the highest during winter months. In general, the median PM2.5 levels in winter months are at least twice as high as that of summer months, except for city Beijing where PM2.5 levels demonstrate a minor irregular change from month to month. 

**The yearly trends reveal a gradual improvement over time**:
* PM2.5 levels follow downward trends in all five cities, indicative of a gradual improvement in air quality over time (except for the minor setback in 2013).

(Note: Day-of-the-week and seasonal trends are not shown, as day-of-the-week trends show only minor fluctuations with no clear trend and no difference between weekdays and weekends, and Seasonal trends are fairly consistent with monthly trends.)

### 3.4 The tale betweeen air quality and weather

The aforementioned analysis suggest that there are many underlying factors playing important roles in shaping the air quality (PM2.5), weather conditions being part of it. Here, meteorological weather measurements, including temperature, dew point, pressure, humidity, wind direction and speed, and precipitations, are examed and correlated to PM2.5 levels.

<img src="figures/correlation_matrix_pm_and_weather.svg">

Correlation matrix is first created to gather a quick grasp on the whole figure. As shown, PM2.5 levels are negatively correlated to temperature, dew point, wind speed and precipitation, and positivley correlated to pressure and humidity. In the following figures, each correlation is analyzed with more details.

<img src="figures/jointplots_combined.svg">

As evident in the jointplots: <br>
**Temperature:** there is a negative correlation between PM2.5 and temperature. In particular, higher PM2.5 levels are mostly associated with lower temperatures below 10 celsius (cold weather). <br>
**Dew point:** Similar to temperature, there is a negative correlation between PM2.5 and dew point. Higher PM2.5 levels are mostly associated with lower dew points below 5 degree celsius. This is expected as temperature and dew point are strongly positively correlated.<br>
**Pressure:** There is a positive correlation between PM2.5 and pressure. In partucular, higher PM2.5 levels are mostly associated with higher atmospheric pressures. This is also expected as pressure and temperature are negatively correlated.<br>
**Humidity:** There is a weak positive correlation between PM2.5 readings and humidity, but it's still statistically significant. Higher PM2.5 levels are more likely to occur at higher humidity levels.<br>

<img src="figures/pm_wind_direction_by_city.svg">

**Wind direction:**<br> 
Wind directions are classified into five categories: northwest(NW), northeast(NE), southeast(SE), southwest (SW) and static wind (CV). Boxplots shwon above reveal a strong dependency of PM2.5 on wind direction. More importantly, the dependence differs by city, as determined by the unique location and neighbour environment of each city.

**Wind speed:**<br> 
As hinted above, the correlation of PM2.5 with wind speed is likely to be dependent on both city and wind direction. Therefore, PM2.5 is plotted as a function of wind speed by city as well as wind direction. 

<img src="figures/pm_wind_speed_combined.svg">

Indeed, the correlation is complicated by the dependency on city and wind direction:<br>
* Beijing: winds from northwest are the strongest, while winds from northeast are most effective at reducing high PM2.5 levels; 
* Chengdu: winds from northeast are the strongest, while winds from southwest are most effective at reducing high PM2.5 levels; 
* Guangzhou: winds from the north (NE, NW) are stronger. Although higher wind speeds also help to reduce PM2.5, the effect is very limited;
* Shanghai: winds from the east (NE, SE) are stronger, while winds from southwest are most effective at reducing high PM2.5 levels;
* Shenyang, winds from the west (NW, SW) are stronger, while winds from the east (NE, SE) are more effective at reducing PM2.5 levels.

Despite the complication, there are some common characteristics shared by all cities:<br>
* Higher wind speeds help to reduce PM2.5 levels, this has been observed with all wind directions in all cities; 
* Static winds feature low wind speeds (<5 m/s), therefore they have limited effect on reducing PM2.5 levels; 
* The wind direction with higher wind speeds has smaller PM2.5 median value and interquantile values;
* The wind direction with a stronger correlation between wind speed and PM2.5 has broader PM2.5 interquantile ranges, this is true because PM2.5 level varies more with wind speed and causes wider distribution of PM2.5;

<img src="figures/pm_precipitation.svg">

**Precipitation**:<br>
As shown above, precipitation helps to reduce high PM2.5 levels regardless of the type of precipitation. In particular, large or extended periods of precipitation (high cumulative precipitation) is very effective at eliminating high PM2.5 readings (>100). However, the impact becomes very limited for low PM2.5 readings (< 50). 
Between rains and snows, rains are more effective at reducing PM2.5 levels than snows.

More details and source codes on EDA are available at: <br>
https://github.com/jingzhaomirror/Springboard-Capstone-Project-1/blob/master/milestone_report.ipynb <br>
https://github.com/jingzhaomirror/Springboard-Capstone-Project-1/blob/master/data_analysis.ipynb

## 4. Modeling
Three different methodologies are implemented to model and predict the air quality (PM2.5) in the above five major Chinese cities.

## 4.1 Time series PM2.5 modeled by ARIMA and SARIMA

First, PM2.5 data by itself is treated as a time series and modeled using ARIMA/SARIMA from Statsmodels. The following procedures are followed for constructing the ARIMA-based models: 
* To use the ARIMA/SARIMA models, Dickey-Fuller Test is first conducted to determine if the time series of interest is stationary (constant mean, variance and autocorrelation across the time series dataset). The resulting p-value is very small (essentially zero), suggesting that the time series meet the stationarity requirement and no transformations are necessary;
* ACF and PACF plots are then graphed to identify the potential parameters ($p$, $d$, $q$, ($s$)) for building ARIMA and SARIMA models. Based on the plots, the following parameters are chosen for ARIMA and grid search is performed on the training set (time series data of year 2010-2014) to identify the best combination:
    * $p$: the number of Auto-Regressive Terms based on the pacf plot is 1 or 2
    * $d$: the number of differences to be performed is 0, given that no data transformation is necessary
    * $q$: The number of Moving Average Terms based on the acf plot: 0 or 1 or 2
    * Grid search results show that ARIMA models of various combinations of $p$, $d$, $q$ share very similar performance, with the best combination for city Beijing being p=2, d=0, q=0.
* ARIMA model is finally constructed using the best combination of $p$, $d$, $q$ for each city, trained on data of year 2010-2014 and tested on data of year 2015. Shown below is an overview of the ARIMA model optimized for city Beijing:  

<img src="figures/arima_model.png">

As shown, ARIMA models demonstrate acceptable performance on training sets, reasonable precision for the very first out-of-sample prediction into the future, but poor predictions for further steps into the future. In addition, SARIMA model is also carried out to best account for the observed seasonal component (yearly). Again, the resulting SARIMA model only provides a reasonable prediction for the very first data point into the future, but fails to predict further steps. This suggests that the autoregressive and moving average patterns in the time series alone are not enough to predict multiple steps into the future. There are other underlying factors causing non-repetitive patterns in PM2.5.

## 4.2 PM2.5 regression models using datetime and weather features

Learning from 4.1, regression models are constructed by incorporating available meteorological weather data (temperature, pressure, dew point, wind direction, wind speed, precipitation). Here, datetime information is represented as year, month, day, hour and season, and used as features along with the weather features. Following this methodology, various regression models are constructed and fine-tuned using data of all five cities from year 2010 to 2014 as the training set, then tested on the data of five cities in year 2015 as the holdout test set. The results obtained from the grid search optimized regression models are summarized below: 

<img src="figures/regressors_without_feature_engineering.png">

As shown, simple linear models (LinearRegression, Lasso and Ridge) only achieves $R^2$=0.21 on the test set. Among non-linear models, KNeighborsRegressor gives $R^2$ of 0.34; ensemble methods, like RandomForestRegressor and GradientBoostingRegressor, achieve $R^2$ of 0.42-0.44; In addition, artificial neural network based MLPRegressor is also tried out with a reported $R^2$ of 0.39. Of all regression models, Gradient Boosting Regressor produces the best performance on the holdout set, the performance of which is plotted in the middle of the above figure. <br>

In addition, the feature importance rankings from the optimized Gradient Boosting Regressor and Random Forest Regressor are also shown in the above figure. Although the ranking varies slightly, the top seven features are consistent. These reveal that among all meteorological parameters, wind speed (ws), temperature (temp), humidity (humi), dew point (dewp) and pressure (pres) are the top influencers; among all datetime-related parameters, month and day are the most important factors.<br>

In addition, separate machine learning models are attempted for individual cities. By separating out the 'city' feature into individual models, $R^2$ is improved from 0.4-0.45 to 0.45-0.5. But, further improvements are hard without additional feature engineering. <br> 

These suggest that although time-related information and weather conditions can only explain the variations in air quality (PM2.5 value) to a limited extent. There are other underlying reasons causing the PM2.5 trends and variations. <br>

## 4.3 Improved regression models by introducing additional lag feature

Given the conclusions from 4.1 and 4.2, and the fact that PM2.5 is intrinsically a time series that progresses on its lagged (prior) values, an additional lag feature of 1 hour (PM2.5 value from the previous one step) is introduced to take into consideration the impact from its previous values.<br>

To implement this, all the regression models from 4.2 are updated by adding the new lag feature (value from the last step in history). In addition, both data of year 2015 and data of city 'shenyang' are reserved as the holdout test set, that is, only data from the other four cities in year 2010-2014 are used as the training set. This allows evaluating not only the model's capability for forecasting into the future but also its ability to predict for an unseen city. <br>

<img src="figures/regressors_with_feature_engineering.png">

As summarized in the figure, in this particular problem, it’s very useful to include the lag feature from previous steps for predicting future steps and R2 scores are boosted significantly regardless of the model used. In fact, simple linear models report a satisfactory $R^2$ score of 0.94 on the test set, whereas more advanced models do not offer further improvement. The feature importance ranking reveals that among all features, the newly introduced lag feature accounts for more than 60% of the contribution, meteorological (weather) and geographic (city) features account for up to 25% and the rest is attributed to the datetime based features. In addition, the good score reported on the test set indicates that the models powered by the additional lag feature not only forecast into the future, but also extends to predict an unseen city. <br>

### 4.4 Summary
Modeling and predicting air quality (PM2.5) is a complex problem, as there are many underlying factors all playing an role in shaping PM2.5. As revealed in this study, both previous observed value and various weather conditions have an impact on future PM2.5 values. As the winning model suggests, previous observation is crucial in setting the baseline of the next step, and various meteorological weather conditions as well as datetime features determines how PM2.5 value in the next step deviates from its previous step. 

More details and source codes on modeling are available at: <br>
https://github.com/jingzhaomirror/Springboard-Capstone-Project-1/blob/master/modeling.ipynb

## 5. Limitation and potential future work

### Limitation
The winning model requires values in previous steps for predicting future step, therefore, its ability to forecast the future is limited, because error propagates as the prediction is used to forecast further steps. In addition, for practical use, pipeline needs to be set up to log the forecasted PM2.5 values for enabling further step predictions. To ensure good accuracy, the dataset and model should be updated whenever a new PM2.5 observation is available.<br>

### Future work
The use of ensembles of the above machine learning models might be beneficial to average out bias and improve performance; In addition, instead of the hourly frequency, it might be of more practical application to downsample the entire dataset on a daily frequency and use the lag feature of previous day for forecasting the next day.<br>

## 6. Conclusion

Understanding and predicting air pollution have become increasingly important to people at all levels in China, due to the severe air problem in the past few years. Hourly PM2.5 data of five major Chinese cities in the period of 2010-2015 are acquired, analyzed and modeled in this project, to identify PM2.5 patterns and forecast future air quality

**Patterns in PM2.5**
* PM2.5 measurements are fairly consistency among nearby stations in all five cities.
* PM2.5 levels vary siginificantly among cities. The rankings of PM2.5 from the lowest to the highest are Guangzhou, Shanghai, Chengdu and Shenyang, Beijing. This corresponds to the air quality from the best to the worse, respectively. Unfortunately, PM2.5 levels for all five cities are too high to be considered healthy and safe according to EPA standards. As the data implies, although PM2.5 levels are gradually improving over time in all five cities, the improvement is only moderate, and there is still a long way to go. 
* Time-dependent trends of PM2.5 reveal that PM2.5 level varies systematically with hour of the day, month/season of the year and from year to year, whereas only minor variation of PM2.5 on day of the week is observed; PM2.5 daily and monthly/seasonal trends are unique with individual cities, but yearly trend from 2010 to 2015 shows a consistent decrease in PM2.5 levels among five cities.
* Correlations with meteorological weather data reveal that all the weather features available in the datasets affect PM2.5 levels to some extent: Both high winds and large precipitations help to reduce high PM2.5 levels in all five cities; The dependence of PM2.5 on wind direction is unique for each city, as determined by location and neighbour environment; PM2.5 levels are also negatively correlated to temperature and dew point, and positively correlated to pressure and humidity.


**Modeling PM2.5** <br>
Three methodologies are implemented to model and predict the air quality (PM2.5 value) for these cities:
* Time series PM2.5 data by itself is first modeled using ARIMA model:<br>
The resulting ARIMA models demonstrate acceptable performance on training sets, good precision for the very first out-of-sample prediction into the future, but poor predictions for further steps into the future. This suggests that the autoregressive and moving average patterns in the time series alone are not enough to predict multiple steps into the future.
* Various regression models are then constructed by incorporating available meteorological weather data in addition to the datetime features: <br>
Neither the simple linear models (LinearRegression, Lasso and Ridge) nor the more advanced models (KNeighborsRegressor, RandomForestRegressor, GradientBoostingRegressor and MLPRegressor) yield satisfying performance, indicating that time-related information and weather conditions can only explain the variations in PM2.5 to a limited extent. The result from the optimized model reveals that among all meteorological parameters, wind speed (ws), temperature (temp), humidity (humi), dew point (dewp) and pressure (pres) are the top influencers; among all datetime-related parameters, month and day are the most important factors.
* Additional lag feature (value from previous steps) is introduced to take into consideration its own history (autoregressive nature):<br>
The above regression models are updated with the new lag feature. The results reveal that, in this particular problem, it’s very useful to include the lag feature from previous steps for predicting future steps and $R^2$ scores are boosted significantly regardless of the model used, and the best $R^2$ score achieved is 0.94. The feature importance ranking reveals that among all features, the newly introduced lag feature accounts for more than 60% of the contribution, meteorological (weather) and geographic (city) features account for up to 25% and the rest is attributed to the datetime based features. The winning model not only forecasts into the future, but also extends to predict an unseen city. <br>